Messaggio che Alice vuole inviare a Bob

In [1]:
import random

In [ ]:
# 'with' è un "gestore di contesto": garantisce che il file venga chiuso automaticamente non appena finiamo di leggere, evitando sprechi di memoria o errori di sistema
# questo comando apre il file "Alice.txt" in modalità lettura
with open("Alice.txt", "r") as file:
    # legge tutto il contenuto del file e rimuove eventuali spazi bianchi iniziali/finali
    input_str = file.read().strip() # stringa di input

print(input_str)

# modifiche

Ciao Bob!


Il messaggio viene prima convertito in codice ASCII, successivamente in codice binario

In [3]:
# converte una stringa in una rappresentazione binaria, dove ogni carattere è rappresentato da 8 bit come lista di numeri
def string_to_binary(s):
    # per ogni carattere 'c' nella stringa, ottiene il codice ASCII con ord(c),
    # format(..., '08b') lo converte in binario con 8 bit con padding di zeri a sinistra, e crea una lista di interi
    # il ciclo più esterno (il primo) itera sui caratteri della stringa
    # il ciclo interno (il secondo) itera sui bit della stringa binaria di quel carattere e li converte in interi
    return [int(b) for c in s for b in format(ord(c), '08b')]

# Converte la stringa letta in rappresentazione binaria
input_bin = string_to_binary(input_str)

print(input_bin)

[0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]


Dividiamo la lista di bit in raggruppamenti di $k$ bit ciascuno, ognuno dei quali sarà completato da un numero $r$ di bit di parità che serviranno per il controllo degli errori secondo

$$ 2^r \ge k + r + 1 $$

più un bit di parità globale.
Nel blocco esteso al bit di parità sono riservate le posizioni indicizzate dalle potenze di $2$ (la parità globale è descritta dal bit in posizione $0$), il messaggio è scritto nei bit rimanenti.

In [18]:
k = 4 # lunghezza del messaggio in bit per ogni blocco

# funzione che calcola le posizioni dei bit di parità
# prende in input la lunghezza del messaggio
def parity_bits(k):
    r = 0
    while (2**r) < (k + r + 1):
        r += 1
    return [2**i for i in range(r)]

# funzione che calcola le posizioni dei bit riservati al messaggio
# prende in input la lunghezza del messaggio
def message_bits(k):
    r = len(parity_bits(k)) # numero dei bit di parità
    data_bits = []
    for i in range(1, k + r + 1):   # l'elemento con indice 0 è riservato al bit di parità globale
        # verifica se 'i' non è una potenza di 2 usando l'operatore bitwise AND (&)
        # se i è potenza di 2 (es. 1, 2, 4, 8), 'i & (i-1)' è sempre 0
        if (i & (i - 1)) != 0:
            data_bits.append(i)
    return data_bits

k = 4 # lunghezza del messaggio in bit per ogni blocco
r = len(parity_bits(k))  # numero dei bit di parità
n = 1 + k + r  # lunghezza del blocco (incluso il bit di parità globale)

sent_data = []

for i in range(0, int(len(input_bin)/k)):
    message = input_bin[i*k:(i+1)*k]
    block = [0] * (1 + k + r)
    # posizioniamo nelle loro posizioni: bit di messaggio, bit di parità, bit di parità globale
    data_idx = 0
    for i in message_bits(k):
        block[i] = message[data_idx]
        data_idx += 1
    for i in parity_bits(k):
        parity = 0
        for j in range(1, k + r + 1):
            if j & i:  # Se il risultato dell'AND è diverso da zero, significa che la posizione 'j' è controllata dal bit di parità corrente
                parity ^= block[j]  # il comando '^' è lo XOR per calcolare la parità
        block[i] = parity
    # calcolo del bit di parità globale
    overall_parity = sum(block) % 2
    block[0] = overall_parity
    sent_data.append(block)

# dividi la lista "input_bin" in sottoliste di 4 bit ciascuna
input_bin_div = [input_bin[i:i+4] for i in range(0, len(input_bin), k)]
print(input_bin_div)
print(sent_data)

[[0, 1, 0, 0], [0, 0, 1, 1], [0, 1, 1, 0], [1, 0, 0, 1], [0, 1, 1, 0], [0, 0, 0, 1], [0, 1, 1, 0], [1, 1, 1, 1], [0, 0, 1, 0], [0, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 1, 1, 0], [1, 1, 1, 1], [0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1]]


Inseriamo degli errori casuali per ogni blocco che viene inviato a Bob

In [19]:
# introduciamo degli errori casuali
p = 0.05  # probabilità di errore (bit flip)
# send = 1
sent_data_with_errors = []
for i in range(len(sent_data)):
    block_with_errors = sent_data[i].copy()
    for j in range(len(sent_data[i])):
        if random.random() < p:
            # Effettua il bit flip
            if sent_data[i][j] == 0:
                block_with_errors[j] = 1
            else:
                block_with_errors[j] = 0
    sent_data_with_errors.append(block_with_errors)

print(sent_data)
print(sent_data_with_errors)

[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1]]
[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 0, 0, 1], [0, 0, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 1, 1, 0, 1, 0], [0, 1, 1, 0, 1, 1, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1], [0, 1, 1, 1, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 1, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 0, 0, 1]]


Proviamo a mandare a video il messaggio ricevuto da Bob con gli errori.

In [14]:
output_bin_with_errors = []

for i in range(len(sent_data_with_errors)):
    message = [sent_data_with_errors[i][j] for j in message_bits(k)]
    output_bin_with_errors.extend(message)
# converte una lista di bit (numeri 0 o 1) in una stringa alfanumerica
def binary_to_string(binary_list):
    # lista per raccogliere i caratteri convertiti
    result = []
    # itera sulla lista binaria a passi di 8 bit
    for i in range(0, len(binary_list), 8):
        # estrae un blocco di 8 bit
        byte = binary_list[i:i+8]
        # se il blocco ha esattamente 8 bit, lo converte
        if len(byte) == 8:
            # converte la lista in stringa binaria, poi in intero, poi in carattere ASCII
            byte_str = ''.join(str(b) for b in byte)
            char = chr(int(byte_str, 2))
            result.append(char)
    # unisce tutti i caratteri in una stringa
    return ''.join(result)

# output_str = binary_to_string(output_bin)
print(output_bin_with_errors)
output_str = binary_to_string(output_bin_with_errors)
print(output_str)

[0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
Kian Rob!


Correggiamo gli errori.

In [20]:
# funzione che calcola la sindrome del blocco ricevuto
def sindrome(block):
    s = 0
    for i in range(1, len(block)):
        if block[i] == 1:
            s ^= i
    return s

# funzione che calcola il bit di parità globale
def global_parity_check(block):
    return sum(block) % 2

# funzione che rileva e corregge gli errori nel blocco ricevuto
def correct_errors(block):
    s = sindrome(block) # calcola la sindrome del blocco ricevuto
    if s == 0:
        return block  # nessun errore rilevato
    else:
        block[s] ^= 1  # Corregge l'errore
        if global_parity_check(block) == 0:
            return block  # errore singolo corretto
        else:
            return 1  # errore multiplo rilevato, chiediamo di reinviare il blocco

print(sent_data)
print(sent_data_with_errors)

sent_data_corrected = []
for i in range(len(sent_data_with_errors)):
    send = 1
    while send == 1:
        send = correct_errors(sent_data_with_errors[i])
    sent_data_corrected.append(send)

print(sent_data_corrected)

[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1]]
[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 0, 0, 1], [0, 0, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 1, 1, 0, 1, 0], [0, 1, 1, 0, 1, 1, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1], [0, 1, 1, 1, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 1, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 0, 0, 1]]
[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1,

In [21]:
output_bin_corrected = []

for i in range(len(sent_data_corrected)):
    message = [sent_data_corrected[i][j] for j in message_bits(k)]
    output_bin_corrected.extend(message)

output_str = binary_to_string(output_bin_corrected)
print(output_str)

Ciao Bob!
